In [1]:
cd("$(homedir())/Documents/Repos/enso_project.jl")
using Pkg
Pkg.activate(".")

  Activating project at `C:\Users\lisah\Documents\Repos\enso_project.jl`


In [2]:
using ReservoirComputing, CSV, DataFrames, DynamicalSystems, Plots, enso_project

┌ Warning: Package cuDNN not found in current path.
│ - Run `import Pkg; Pkg.add("cuDNN")` to install the cuDNN package, then restart julia.
│ - If cuDNN is not installed, some Flux functionalities will not be available when running on the GPU.
└ @ FluxCUDAExt C:\Users\lisah\.julia\packages\Flux\9PibT\ext\FluxCUDAExt\FluxCUDAExt.jl:10


In [4]:
data = Matrix(CSV.read("data/sst_data/sst_34_anomaly_embedded.txt", DataFrame; delim=',', ignorerepeated=true, header=false));

In [5]:
spectral_radii = [0.8, 0.9, 1.0]
sparsities = [0.05, 0.08,0.1, 0.12, 0.14, 0.2]
input_scales = [0.1, 0.5, 1.0, 2.0]
ridge_values = [0.0, 1e-6, 1e-5]
reservoir_sizes = [90, 110, 120, 125, 130, 135, 140, 150, 160, 180, 190, 200, 210]
param_grid = enso_project.create_param_grid(reservoir_sizes, spectral_radii, sparsities, input_scales, ridge_values);

In [6]:
data_train = data[1:395,:]'; # train data is kept fixed to 80%

5×395 adjoint(::Matrix{Float64}) with eltype Float64:
  0.08  -0.2   -0.14   0.02   0.49  …  -0.03  -0.01   0.31   0.43   0.75
  0.27   0.86   1.24   1.73   1.68      0.51   0.75   0.44   0.82   0.83
  2.13   1.81   1.22   0.68   0.68      1.26   1.65   1.79   2.21   2.72
 -0.33  -0.44  -0.38  -1.02  -1.29      2.47   2.23   1.61   0.98   0.27
 -0.96  -0.16  -0.52  -0.54  -0.69     -0.42  -0.49  -0.43  -0.7   -0.64

In [ ]:
# sample size N
N = 20

# lead time L
L = 30

# store predictions
predictions = zeros(N,L)

# store test data 
datasets_test = zeros(N,L);

In [10]:
for n in 1:N
    println("n=$n")
    val_data = data[396:395+n+48 , :]' # start validation set of length 48+n
    test_data = data[395+n+49:395+n+48+L,:]' # test against L month forecast lead
    datasets_test[n,:] = test_data[1,:]
    esn, W_out, val_loss = enso_project.cross_validate_esn(data_train, val_data, param_grid) 
    prediction = enso_project.esn_eval_pred(esn, W_out, test_data)
    sse_vals[n,:] .= sse(prediction, test_data[1,:])
    predictions[n,:] = prediction
end

n=1
enso_project.ESNHyperparams(90, 0.8, 0.05, 0.1, 0.0)
Validation loss = 165.89071796388623
enso_project.ESNHyperparams(150, 0.8, 0.05, 0.1, 0.0)
Validation loss = 125.74689246733459
enso_project.ESNHyperparams(110, 0.9, 0.05, 0.1, 0.0)
Validation loss = 102.28493453843394
enso_project.ESNHyperparams(180, 0.9, 0.05, 0.1, 0.0)
Validation loss = 81.93028582436975
enso_project.ESNHyperparams(90, 1.0, 0.1, 0.1, 0.0)
Validation loss = 74.17290271260012
enso_project.ESNHyperparams(90, 0.9, 0.12, 0.1, 0.0)
Validation loss = 54.501160344104136
enso_project.ESNHyperparams(130, 0.9, 0.14, 0.1, 0.0)
Validation loss = 30.50979942909385
enso_project.ESNHyperparams(130, 0.9, 0.14, 0.1, 1.0e-6)
Validation loss = 30.49596650913812
enso_project.ESNHyperparams(130, 0.9, 0.14, 0.1, 1.0e-5)
Validation loss = 30.42386160998865
n=2
enso_project.ESNHyperparams(90, 0.8, 0.05, 0.1, 0.0)
Validation loss = 171.4474097332627
enso_project.ESNHyperparams(150, 0.8, 0.05, 0.1, 0.0)
Validation loss = 128.41675444018

In [11]:
CSV.write("C:/Users/lisah/Documents/Repos/enso_project.jl/error_saves/enso_L$(L)_N$(N)_predictions.csv", DataFrame(predictions, :auto))
CSV.write("C:/Users/lisah/Documents/Repos/enso_project.jl/error_saves/esn_L$(L)_N$(N)_testsets.csv", DataFrame(datasets_test, :auto))

"C:/Users/lisah/Documents/Repos/enso_project.jl/error_saves/esn_L30_N20_testsets.csv"

In [ ]:
function rmse(predictions::AbstractMatrix, test_data::AbstractMatrix)
    N, L = size(predictions, 1), size(predictions,2)
    sse_vals = zeros(N,L)
    for n in 1:N
        sse_vals[n, :] = sum.((predictions[n,:] .- test_data[n,:]).^2)
    end
    rmse = sqrt.(sum(sse_vals, dims=1) ./N)
    return rmse
end

true

In [ ]:
function corr(predictions::AbstractMatrix, test_data::AbstractMatrix)
    p_bar, t_bar = mean(predictions, dims=1), mean(test_data,dims=1) # compute sample mean across i in 1:N for each lead time
    p_denom = 0
end